<a href="https://colab.research.google.com/github/hopkinslaurel/SI_data_augmentation/blob/main/Sentinel_2_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount drive to collab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import EE and geemap libraries
import ee
import geemap.core as geemap

In [3]:
# Authenticate and initialize EE service
project_id = 'ee-hopkilau'  # Google Cloud project ID

ee.Authenticate()
ee.Initialize(project=project_id)

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [20]:
# User defined parameters
cloud_percentage = 20  # filter granules to have low cloud coverage

In [42]:
# GEE code to process Sentinel-2 data https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED#description

def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [43]:
# Add raster
jan_2023_climate = (
    ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')
    .filterDate('2023-01', '2023-02')
    .first()
)

sentinel_2020_jan = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')  # L1C TOA reflectance
    .filterDate('2020-07-01', '2020-07-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage))
    .map(mask_s2_clouds)
)
sentinel_2020_jan

/usr/local/lib/python3.10/dist-packages/eerepr/repr.py:57: UserWarning: Getting info failed with: 'Collection query aborted after accumulating over 5000 elements.'. Falling back to string repr.
  warn(f"Getting info failed with: '{e}'. Falling back to string repr.")


In [44]:
# Instantiate a map object
#m = geemap.Map(center=[30, 0], zoom=2)

rgb_vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(-9.1695, 38.6917, 12)
m.add_layer(sentinel_2020_jan.median(), rgb_vis, 'RGB')
m

Map(bottom=402197.0, center=[38.6917, -9.1695], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…